# Example of How to Use GenAI with ConvoKit genai Module

The ConvoKit GenAI module provides a unified interface for working with large language models (LLMs) while doing conversational analysis in ConvoKit. It supports multiple providers including OpenAI GPT, Google Gemini, and local models through a simple factory pattern. This module makes it easy to integrate AI-powered text generation into your ConvoKit workflows for diverse tasks. The module handles API key management, response formatting, and provides consistent interfaces across different LLM providers.


## Setup config for GenAI with GPT

Setting up config info to access models is mandatory but simple. For models we implemented (GPT and Gemini), we provide methods to set API keys so they are stored in the environment. For other models or local LLMs, users can also implement them in similar manner. Here we provide a simple demonstration with configuring for OpenAI's GPT model.

In [1]:
from convokit.genai.genai_config import GenAIConfigManager

In [ ]:
config = GenAIConfigManager()
config.set_api_key("gpt", "YOUR API KEY")
print(f"Successfully set OpenAI API key in config.")

Successfully set OpenAI API key in config.


## Initialize clients to Communicate with models

After setting the API key, we are ready to communicate with the models. Retrieve response the same as you would normally do interacting with models through API. However, we do wrap the LLM responses in a unified class, so we can handle all LLM response format easily. Users are expected to follow similar template when implementing clients for other models.

In [ ]:
from convokit.genai import get_llm_client

MODEL_PROVIDER = "gpt"
client = get_llm_client(MODEL_PROVIDER, config)

In [4]:
response = client.generate([{"role": "user", "content": "Tell me a fun fact about Cornell University."}])
print("Text:", response.text)
print("Tokens:", response.tokens)
print("Latency:", response.latency)

Text: A fun fact about Cornell University is that it is home to the world's first university-based hotel management program. Established in 1922, the School of Hotel Administration at Cornell has been a leader in hospitality education, shaping the future of the industry and producing many influential leaders in hospitality and tourism. The program is renowned for its rigorous curriculum and strong connections to the hospitality industry.
Tokens: 90
Latency: 2.0230631828308105


# Setup ConvoKit GenAI with Google Gemini Through Vertex AI

Very similar to GPT. Checkout Vertex AI: https://cloud.google.com/vertex-ai/generative-ai/docs/start/quickstarts/quickstart-multimodal

In [ ]:
MODEL_PROVIDER = "gemini"
MODEL = "gemini-2.0-flash-001"
config.set_google_cloud_config("YOUR PROJECT", "YOUR LOCATION")

In [7]:
client = get_llm_client(MODEL_PROVIDER, config)
response = client.generate("Tell me a fun fact about Cornell University.")
print("Text:", response.text)

Text: Here's a fun fact about Cornell University:

Cornell has a tradition called "Slope Day," a massive student-organized festival held on the last day of classes each spring. It's a day of music, food, and general celebration on Libe Slope, a large grassy hill on campus. While it's a beloved tradition, it's also known for its, shall we say, *spirited* atmosphere!



# Using GenAITransformer with ConvoKit

The GenAITransformer provides a powerful way to apply LLM processing to different levels of ConvoKit objects (utterances, conversations, speakers, or corpus). Let's demonstrate this with the friends corpus.


In [ ]:
# Load the friends corpus and get the first conversation
from convokit import Corpus, download
from convokit.genai import GenAITransformer

# Load the corpus
corpus = Corpus(filename=download("friends-corpus"))

# Get the first conversation and its first two utterances
first_convo = list(corpus.iter_conversations())[0]
utterances = first_convo.get_chronological_utterance_list()[:2]

print(f"First conversation ID: {first_convo.id}")
print(f"Number of utterances in first conversation: {len(first_convo.get_chronological_utterance_list())}")
print(f"We'll process the first 2 utterances:")
for i, utt in enumerate(utterances):
    print(f"  Utterance {i+1}: {utt.speaker.id}: {utt.text[:100]}...")


## Example: Sentiment Analysis on Utterances

Let's create a GenAI transformer that analyzes the sentiment of utterances and stores the result as metadata.


In [ ]:
# Create a sentiment analysis transformer
sentiment_transformer = GenAITransformer(
    provider="gpt",  # Use the same provider we configured earlier
    model="gpt-4o-mini",
    object_level="utterance",  # Process at utterance level
    prompt="Analyze the sentiment of the following text and respond with just one word: 'positive', 'negative', or 'neutral'. Text: {formatted_object}",
    formatter=lambda utterance: utterance.text,  # Extract text from utterance
    metadata_name="sentiment",  # Store result in 'sentiment' metadata field
    selector=lambda utterance: utterance.id in [utt.id for utt in utterances],  # Only process our 2 utterances
    config_manager=config
)

# Apply the transformer to the corpus
corpus = sentiment_transformer.transform(corpus)

print("Sentiment analysis results:")
for utt in utterances:
    sentiment = utt.meta.get("sentiment", "Not processed")
    print(f"  {utt.speaker.id}: '{utt.text[:50]}...' -> Sentiment: {sentiment}")
